# Step 2: Model Building & Evaluation

In [1]:
# import the libraries
import tensorflow as tf
from tensorflow import keras
from keras import layers
import h5py
import os
import pandas as pd
import numpy as np
import csv

import urllib
import glob
import pickle
import re

from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.optimizers import SGD

# https://www.intel.com/content/www/us/en/developer/articles/guide/guide-to-tensorflow-runtime-optimizations-for-cpu.html
#tf.config.threading.set_inter_op_parallelism_threads(70) 
#tf.config.threading.set_intra_op_parallelism_threads(70)
#tf.config.set_soft_device_placement(True)

print('Import complete successfully!')

Import complete successfully!


## Load feature data set

Load the data and dump a short summary of the resulting DataFrame.

In [ ]:
train_df = pd.read_csv('./data/intermediateData/PdM_telemetry_model_age_16_maint_err_fail_RUL_cycle_train.csv', sep=",", header=0)
train_df.loc[train_df['failures'] == '-', 'failures'] = 0
for fail in train_df['failures']:
    if fail != 0:
        train_df.loc[train_df['failures'] == fail, 'failures'] = int(fail)
train_df_sorted = train_df.sort_values(by=['machineID', 'datetime'], ascending=[True, False])
for index, row in train_df_sorted.iterrows():
    if row['failures'] != 0 and row['label1'] == 1:
        failures_prev = row['failures']
    elif row['failures'] == 0 and row['label1'] == 1:
        train_df_sorted.loc[train_df_sorted.index == index, 'failures'] = failures_prev
train_df = train_df_sorted.sort_values(by=['machineID', 'datetime'], ascending=[True, True])
train_df.to_csv('./data/intermediateData/PM_train_pre_prepared.csv')
train_df

In [ ]:
test_df = pd.read_csv('./data/intermediateData/PM_test_almost_prepared.csv', sep=",", header=0)
test_df.loc[test_df['failures'] == '-', 'failures'] = 0
for fail in test_df['failures']:
    if fail != 0:
        test_df.loc[train_df['failures'] == fail, 'failures'] = int(fail)
test_df_sorted = test_df.sort_values(by=['machineID', 'datetime'], ascending=[True, False])
for index, row in test_df_sorted.iterrows():
    if row['failures'] != 0 and row['label1'] == 1:
        failures_prev = row['failures']
    elif row['failures'] == 0 and row['label1'] == 1:
        test_df_sorted.loc[test_df_sorted.index == index, 'failures'] = failures_prev
test_df = test_df_sorted.sort_values(by=['machineID', 'datetime'], ascending=[True, True])
test_df.to_csv('./data/intermediateData/PM_test_pre_prepared.csv')
test_df

In [ ]:
train_df = pd.read_csv('./data/intermediateData/PM_train_pre_prepared.csv', sep=",", header=0)
train_df

In [ ]:
#balancing training data
#print('Balancing train data is in progress...')
df_train = train_df[["datetime","machineID","cycle","model","age","volt","rotate","pressure","vibration","RUL","label1","label2","cycle_norm"]]
y_train = train_df["failures"]
#print(df_train.shape)
#print(y_train.shape)
#rus = RandomUnderSampler(sampling_strategy={0:30000})
rus = RandomUnderSampler(sampling_strategy={0:19935})
#rus = RandomUnderSampler()
df_resampled, y_resampled = rus.fit_resample(df_train, y_train)
#print(df_resampled.shape)
#print(y_resampled.shape)
df_balanced = pd.concat([df_resampled, y_resampled],axis=1)
train_df = df_balanced.sort_values(['machineID', 'datetime'], ascending=[True, True])
train_df = train_df[['datetime','machineID', 'cycle', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration', 'failures', 'RUL', 'cycle_norm', 'label1', 'label2']]
#check balanced data
#train_df.to_csv('./data/PM_train_prepared_balanced_datetime.csv',index=False)
#print(train_df.shape)
#train_df.drop(train_df.columns[[0]], axis=1, inplace=True)
#check balanced data
#train_df.to_csv('./data/intermediateData/PM_train_prepared_balanced.csv',index=False)
train_df
#print('Balancing train data completed successfully.')

In [ ]:
# filtering out the following categories with their predecessor data rows (failure 0-s)
dfb1 = train_df
dfb1 = dfb1.sort_values(['machineID', 'datetime'], ascending=[False, False])
dfb1.to_csv('./data/intermediateData/PM_train_prepared_1_sorted.csv',index=False)
with open("data/intermediateData/PM_train_prepared_1_sorted.csv", "r", newline='') as sourceb1:
    readerb1 = csv.reader(sourceb1)
      
    with open("data/intermediateData/PM_train_prepared_balanced_2_filtered_unsorted.csv", "w", newline='') as resultb1:
        writerb1 = csv.writer(resultb1)
        failprev=False
        failprevzero=False
        for rb1 in readerb1:
            if rb1[0] == 'datetime':
                writerb1.writerow(rb1)
            elif rb1[9] in (str(1),str(2),str(4)) and failprev == False:
                writerb1.writerow(rb1)
                failprev = True
            elif rb1[9] in (str(1),str(2),str(4)) and failprev == True:
                writerb1.writerow(rb1)
            elif rb1[9] == str(0) and failprev == True and failprevzero == False:
                writerb1.writerow(rb1)
                failprevzero = True
            elif rb1[9] == str(0) and failprev == True and failprevzero == True:
                writerb1.writerow(rb1)
            
sourceb1.close()
resultb1.close()

#print('Writing of .../data/intermediateData/PM_train_prepared_balanced_2_filtered_unsorted.csv completed successfully.')

dfb2 = pd.read_csv('./data/intermediateData/PM_train_prepared_balanced_2_filtered_unsorted.csv', sep=",", header=0)
dfb2 = dfb2.sort_values(['machineID', 'datetime'], ascending=[True, True])
dfb2.drop(train_df.columns[[0]], axis=1, inplace=True)
dfb2.to_csv('./data/preparedData/PM_train_prepared.csv',index=False)
train_df = dfb2
train_df

#print('Writing of .../data/intermediateData/PM_train_prepared_balanced_3_filtered_sorted.csv completed successfully.')

In [ ]:
# filtering out the following categories with their predecessor data rows (failure 0-s)
dfb3 = test_df
dfb3 = dfb3.sort_values(['machineID', 'datetime'], ascending=[False, False])
dfb3.to_csv('./data/intermediateData/PM_test_almost_prepared_1_sorted.csv',index=False)
with open("data/intermediateData/PM_test_almost_prepared_1_sorted.csv", "r", newline='') as sourceb3:
    readerb3 = csv.reader(sourceb3)
      
    with open("data/intermediateData/PM_test_almost_prepared_balanced_2_filtered_unsorted.csv", "w", newline='') as resultb3:
        writerb3 = csv.writer(resultb3)
        failprev=False
        failprevzero=False
        for rb3 in readerb3:
            if rb3[0] == 'datetime':
                writerb3.writerow(rb3)
            elif rb3[9] in (str(1),str(2),str(4)) and failprev == False:
                writerb3.writerow(rb3)
                failprev = True
            elif rb3[9] in (str(1),str(2),str(4)) and failprev == True:
                writerb3.writerow(rb3)
            elif rb3[9] == str(0) and failprev == True and failprevzero == False:
                writerb3.writerow(rb3)
                failprevzero = True
            elif rb3[9] == str(0) and failprev == True and failprevzero == True:
                writerb3.writerow(rb3)
            
sourceb3.close()
resultb3.close()

#print('Writing of .../data/intermediateData/PM_train_prepared_balanced_2_filtered_unsorted.csv completed successfully.')

dfb4 = pd.read_csv('./data/intermediateData/PM_test_almost_prepared_balanced_2_filtered_unsorted.csv', sep=",", header=0)
dfb4 = dfb4.sort_values(['machineID', 'datetime'], ascending=[True, True])
dfb4.drop(test_df.columns[[0]], axis=1, inplace=True)
#test_df.drop(test_df.columns[[0]], axis=1, inplace=True)
dfb4.to_csv('./data/preparedData/PM_test_prepared.csv',index=False)
test_df = dfb4
test_df

In [2]:
train_df = pd.read_csv('./data/preparedData/PM_train_prepared.csv', sep=",", header=0)
print(train_df.head(10))
test_df = pd.read_csv('./data/preparedData/PM_test_prepared.csv', sep=",", header=0)
test_df.loc[test_df['failures'] == '-', 'failures'] = 0
print(test_df.head(10))

   machineID  cycle     model  age      volt    rotate  pressure  vibration  \
0          1     17  0.666667  0.9  0.342238  0.550659  0.319269   0.440007   
1          1     19  0.666667  0.9  0.524516  0.625741  0.337727   0.387953   
2          1     24  0.666667  0.9  0.531867  0.642028  0.313270   0.605337   
3          1     67  0.666667  0.9  0.620057  0.534940  0.390426   0.566431   
4          1     68  0.666667  0.9  0.453993  0.590838  0.489264   0.702829   
5          1     69  0.666667  0.9  0.430023  0.604926  0.470232   0.689213   
6          1     70  0.666667  0.9  0.600360  0.661463  0.463462   0.693415   
7          1     71  0.666667  0.9  0.622540  0.441451  0.410613   0.561752   
8          1     72  0.666667  0.9  0.464634  0.585275  0.389755   0.703889   
9          1     73  0.666667  0.9  0.428900  0.557030  0.344969   0.538842   

   failures  RUL  cycle_norm  label1  label2  
0         0   80    0.002470       0       0  
1         0   78    0.002778       0

c:\Users\bordasm\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## Modelling

In [3]:
# pick a large window size of 50 cycles
sequence_length = 120

In [4]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [5]:
# pick the feature columns 
sequence_cols = ['cycle_norm']
key_cols = ['machineID', 'cycle']
label_cols = ['failures', 'label1', 'label2','RUL']

input_features = test_df.columns.values.tolist()
sensor_cols = [x for x in input_features if x not in set(key_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(label_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols.extend(sensor_cols)

print(sensor_cols)
print(sequence_cols)

['model', 'age', 'volt', 'rotate', 'pressure', 'vibration']
['cycle_norm', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration']


In [6]:
# generator for the sequences
#sequence_cols = train_df.columns.tolist()
seq_gen = (list(gen_sequence(train_df[train_df['machineID']==id], sequence_length, sequence_cols)) 
           for id in train_df['machineID'].unique())

# generate sequences and convert to numpy array
#print(list(seq_gen))
#with open('seq_gen_list.txt', 'w') as f:
#    for item in seq_gen:
#        f.write("%s\n" % item)
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(24717, 120, 7)

In [7]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [8]:
# generate labels
label_gen = [gen_labels(train_df[train_df['machineID']==id], sequence_length, ['failures']) 
             for id in train_df['machineID'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array_orig = label_array
print(type(label_array))
print(label_array.shape)
#print(label_array)

# creating one hot encoder object 
onehotencoder = OneHotEncoder()
df_label_array = pd.DataFrame(label_array)
#reshape the 1-D label_array and finally fit the object 
X = onehotencoder.fit_transform(df_label_array.values.reshape(-1,1)).toarray()
df_label_array = pd.DataFrame(X) 
#printing to verify
#df_label_array.to_csv('./data/df_label_array.csv',index=False)
label_array = df_label_array.to_numpy()
print(type(label_array))
print(label_array.shape)
#print(label_array)

<class 'numpy.ndarray'>
(24717, 1)
<class 'numpy.ndarray'>
(24717, 4)


## Transformer model

In [ ]:
# If theres is an existing model.
model = keras.models.load_model('models/machineErrorTypePredictor_for_type_1_2_4.h5')
print("Model loaded")

In [9]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [10]:
# build the model

input_shape = seq_array.shape[1:]
n_classes = label_array.shape[1]
#nb_features = seq_array.shape[2]
#nb_out = label_array.shape[1]
head_size=1024
num_heads=8
ff_dim=8
num_transformer_blocks=6
mlp_units=[256]
mlp_dropout=0.4
dropout=0.25

inputs = keras.Input(shape=input_shape)
x = inputs
for _ in range(num_transformer_blocks):
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
for dim in mlp_units:
    x = layers.Dense(dim, activation="relu")(x)
    x = layers.Dropout(mlp_dropout)(x)
outputs = layers.Dense(n_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)

#lr_schedule = 1e-4
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=1000,
    decay_rate=0.9)

model.compile(loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["CategoricalAccuracy"])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 7)]     0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 120, 7)      253959      ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 120, 7)       0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 120, 7)      14          ['dropout[0][0]']            

In [11]:
print(seq_array.shape)
print(label_array.shape)

(24717, 120, 7)
(24717, 4)


In [12]:
%%time
# fit the network
earystopping = [keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', # Monitor the validation categorical accuracy
                                                     min_delta=0,    # until it doesn't change (or gets worse)
                                                     patience=5,  # patience > 1 so it continutes if it is not consistently improving
                                                     verbose=0, 
                                                     mode='auto',
                                                     restore_best_weights=True)]

modelcheckpoint = (keras.callbacks.ModelCheckpoint('machineErrorTypePredictor_for_type_1_2_4.h5',
                                    monitor='val_categorical_accuracy',
                                    save_best_only=True,
                                    save_weights_only=False))

model.fit(seq_array, # Training features
          label_array, # Training labels
          validation_split=0.3,
          epochs=20,
          batch_size=32,
          verbose=1,
          callbacks=[earystopping,modelcheckpoint])

Epoch 1/20
541/541 [==============================] - 22981s 42s/step - loss: 1.0829 - categorical_accuracy: 0.5522 - val_loss: 0.9178 - val_categorical_accuracy: 0.6133
Epoch 2/20
541/541 [==============================] - 22824s 42s/step - loss: 0.9136 - categorical_accuracy: 0.6316 - val_loss: 0.8850 - val_categorical_accuracy: 0.6413
Epoch 3/20
541/541 [==============================] - 22494s 42s/step - loss: 0.9327 - categorical_accuracy: 0.6252 - val_loss: 0.8837 - val_categorical_accuracy: 0.6482
Epoch 4/20
541/541 [==============================] - 22464s 42s/step - loss: 0.9025 - categorical_accuracy: 0.6326 - val_loss: 0.9682 - val_categorical_accuracy: 0.6134
Epoch 5/20
541/541 [==============================] - 22342s 41s/step - loss: 0.9889 - categorical_accuracy: 0.5909 - val_loss: 0.9448 - val_categorical_accuracy: 0.6166
Epoch 6/20
541/541 [==============================] - 22258s 41s/step - loss: 0.9357 - categorical_accuracy: 0.6155 - val_loss: 0.9686 - val_categoric

In [13]:
# Save the model
model.save('models/machineErrorTypePredictor_for_type_1_2_4.h5')
print("Model saved")

Model saved


In [14]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Training Accurracy: {}'.format(scores[1]))
# run_logger.log("Training Accuracy", scores[1])

124/124 [==============================] - 11270s 91s/step - loss: 0.8423 - categorical_accuracy: 0.6699
Training Accurracy: 0.6699437499046326


In [15]:
# make predictions and compute confusion matrix
y_pred = (model.predict(seq_array,verbose=1, batch_size=200) > 0.5).astype("int32")
y_pred = [np.argmax(i) for i in y_pred]
y_true = label_array_orig
print('Training Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

124/124 [==============================] - 11240s 91s/step
Training Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[13181,     0,   255,    73,     0],
       [ 2878,     0,     9,   220,     0],
       [ 2912,     0,  2040,     0,     0],
       [    0,     0,     0,     0,     0],
       [ 2231,     0,     0,   918,     0]], dtype=int64)

In [16]:
# compute precision and recall
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = 2 * (precision * recall) / (precision + recall)
print( 'Training Precision: ', precision, '\n', 'Training Recall: ', recall, '\n', 'Training F1 Score:', f1)

Training Precision:  0.5171723131074082 
 Training Recall:  0.6158109802969616 
 Training F1 Score: 0.5621978558221374


c:\Users\bordasm\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bordasm\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model testing

In [17]:
seq_array_test_last = [test_df[test_df['machineID']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['machineID'].unique() if len(test_df[test_df['machineID']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(8, 120, 7)

In [18]:
y_mask = [len(test_df[test_df['machineID']==id]) >= sequence_length for id in test_df['machineID'].unique()]

label_array_test_last = test_df.groupby('machineID')['failures'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(8, 120, 7)
(8, 1)


In [19]:
# test metrics
# creating one hot encoder object 
onehotencoder = OneHotEncoder()
df_label_array_test_last = pd.DataFrame(label_array_test_last)
#reshape the 1-D label_array and finally fit the object 
X = onehotencoder.fit_transform(df_label_array_test_last.values.reshape(-1,1)).toarray()
df_label_array_test_last = pd.DataFrame(X) 
#printing to verify
#df_label_array_test_last.to_csv('./data/df_label_array_test_last.csv',index=False)
#print(df_label_array_test_last.head())
label_array_test_last = df_label_array_test_last.to_numpy()
#print(type(label_array_test_last))
#print(label_array_test_last.shape)
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Test Accurracy: {}'.format(scores_test[1]))

1/1 - 4s - loss: 1.1927 - categorical_accuracy: 0.2500 - 4s/epoch - 4s/step
Test Accurracy: 0.25


In [20]:
# make predictions and compute confusion matrix
y_pred_test = (model.predict(seq_array_test_last,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test.argmax(axis=1), y_pred_test.argmax(axis=1))
cm

1/1 [==============================] - 3s 3s/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[1, 0, 0, 1],
       [1, 0, 0, 0],
       [2, 0, 0, 0],
       [3, 0, 0, 0]], dtype=int64)

In [21]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test, average='weighted')
recall_test = recall_score(y_true_test, y_pred_test, average='weighted')
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Test Precision: ', precision_test, '\n', 'Test Recall: ', recall_test, '\n', 'Test F1 Score:', f1_test)

Test Precision:  0.05 
 Test Recall:  0.125 
 Test F1 Score: 0.07142857142857144


c:\Users\bordasm\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
